In [ ]:
pip install pyldpc

In [ ]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message
from utils import *
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import json
from statsmodels.tsa.ar_model import AutoReg
import numpy as np
from sympy import Matrix
import json
import argparse
from math import log

In [ ]:
def encode_custom(tG, v, seed=None):
    n, k = tG.shape
    rng = check_random_state(seed)
    d = binaryproduct(tG, v)
    x = (-1) ** d
    y = x
    return y


def generate_random_symbols(n):
    # Generate n random bits as -1 or 1 using uniform sampling.
    return np.random.choice([-1, 1], size=n)


def transmit_data(data, channel, snr_db):
    # Apply channel gain to the transmitted data.
    signal = channel * data
    # Calculate the signal power.
    signal_power = np.mean(np.abs(signal)**2)
    # Convert SNR from dB to linear scale and compute noise power.
    noise_power = signal_power / (10**(snr_db / 10))
    # Generate complex Gaussian noise.
    noise = np.sqrt(noise_power / 2) * (np.random.randn(len(data)) + 1j * np.random.randn(len(data)))
    # Add noise to the signal to create the received signal.
    return signal + noise


def transmit_data_awgn(data, snr_db):
    # Apply channel gain to the transmitted data.
    signal = data
    # Calculate the signal power.
    signal_power = np.mean(np.abs(signal)**2)
    # Convert SNR from dB to linear scale and compute noise power.
    noise_power = signal_power / (10**(snr_db / 10))
    # Generate complex Gaussian noise.
    noise = np.sqrt(noise_power / 2) * (np.random.randn(len(data)) + 1j * np.random.randn(len(data)))
    # Add noise to the signal to create the received signal.
    return signal + noise


def estimate_channel(received, data):
    # Reshape the transmitted data for matrix operations.
    X = data.reshape(-1, 1)
    # Perform least squares estimation to solve for the channel.
    h = np.linalg.lstsq(X, received, rcond=None)[0][0]
    return h


def number_of_symbols(N, n, k):
    T = N // (k)
    #This is the number of symbols to be transmitted
    N1 = T*n
    return int(N1)


def decoder_custom_awgn(received):
    list1 = np.sign(np.real(received))
    list2 = []
    for i in list1:
        if i == -1:
            list2.append(1)
        if i == 1:
            list2.append(0)
    return list2


def decoder_custom_jakes(received, channel):
    list1 = np.sign(np.real(received / channel))
    list2 = []
    for i in list1:
        if i == -1:
            list2.append(1)
        if i == 1:
            list2.append(0)
    return list2


def generate_channels_for_doppler_frequencies(Fd, Fs, N):
    def jakes_sos(P, K, Fs, Fd, N, typ):
        # Create a time vector for sampling points
        t = np.linspace(0, P / Fs, P)
        # Calculate angular Doppler frequency
        omega_d = 2 * np.pi * Fd
        # Initialize an array to store Jakes random variables
        jakes_rvs = np.zeros((K, P), dtype=complex)
        # Loop through independent processes
        for k in range(K):
            # Generate random angles for sinusoids
            alpha = np.random.uniform(0, 2 * np.pi, N)
            # Adjust angles based on the number of sinusoids
            alpha_m = np.array([((2 * np.pi * n) - np.pi + al) / (4 * N) for n, al in enumerate(alpha)])
            # Generate random phase offsets for cosine terms
            a_m = np.random.uniform(0, 2 * np.pi, N)
            # Generate random phase offsets for sine terms
            b_m = np.random.uniform(0, 2 * np.pi, N)
            phi_m = np.random.uniform(-np.pi, np.pi, N)
            # Compute cosine terms for the real part
            cosine_terms = np.cos((omega_d * t[:, None] * np.cos(alpha_m)) + phi_m)
            # Sum the cosine contributions to form the real part
            real_part = np.sqrt(2 / N) * np.sum(cosine_terms, axis=1)
            # Check if generating complex signals
            if typ == 'comp':
                # Compute sine terms for the imaginary part
                sine_terms = np.sin((omega_d * t[:, None] * np.cos(alpha_m)) + phi_m)
                # Sum the sine contributions to form the imaginary part
                imag_part = np.sqrt(2 / N) * np.sum(sine_terms, axis=1)
                # Combine real and imaginary parts
                jakes_rvs[k] = real_part + 1j * imag_part
            else:
                # Enforce complex format with zero imaginary part for real signals
                jakes_rvs[k] = real_part + 1j * 0
        # Return the generated Jakes random variables
        return jakes_rvs

    # Number of time points for the output
    P = N
    # Number of independent processes to simulate
    K = 1
    # Specify the type of random variables (complex)
    typ = 'comp'
    # Generate Jakes random variables using the SoS method
    jakes_rvs = jakes_sos(P, K, Fs, Fd, 100, typ)
    correlated_jakes_rvs = jakes_rvs
    # Open a file to write the generated channels
    with open("channels.txt", "w") as fo:
        # Iterate through each sample
        for i in range(P):
            # Extract the i-th sample
            new_sample = correlated_jakes_rvs[:, i]
            # Serialize complex numbers as (real, imag) pairs
            new_sample_serialized = [(z.real, z.imag) for z in new_sample]
            # Write serialized sample to the file
            json.dump(new_sample_serialized, fo)
            # Add a newline after each serialized sample
            fo.write('\n')
    # Return the generated Jakes random variables
    return correlated_jakes_rvs


In [ ]:
def corr_coded(Fd, Fs, N, SNR, n, d_v, d_c):
    H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
    k = G.shape[1]
    rate = k/n
    snr_coded = SNR + 10 * np.log10(rate)
    # N is the number of message bits we want to transmit
    # But we want to send more than N symbols, and k is a fixed value, so we need to compute the number of symbols, and channel values, to generate
    N1 = number_of_symbols(N, n, k)
    
    # Initialize variables for simulation.
    current_index = 0  # Track the current position in channel values.
    total_packets = 0  # Total number of transmitted data bits.
    wrong_packets = 0  # Count of incorrectly received data bits.
    
    generate_channels_for_doppler_frequencies(Fd, Fs, N1)
    with open("channels.txt", "r") as f:
        channel_vals = [json.loads(line)[0] for line in f]
    channel_vals = np.array([complex(re, im) for re, im in channel_vals])
    
    while current_index < N1 - n:
        packet_message = np.random.randint(2, size=k)
         #The encoded packet is n symbols long
        encoded_packet_symbols = encode_custom(G, packet_message) 
        channel = channel_vals[current_index:current_index + n]
        current_index = current_index + n
        received_symbols = transmit_data(encoded_packet_symbols, channel, snr_coded)
        estimated_bits = decode(H, received_symbols.real, snr)
        recovered_message = get_message(G, estimated_bits)
        
        total_packets += len(recovered_message)
        for i in range(len(packet_message)):
            if packet_message[i] != recovered_message[i]:
                wrong_packets += 1
                
    #Return the BER            
    if total_packets == 0:
        return None  # Return None if no packets were transmitted.
    else:
        return np.float16(wrong_packets / total_packets)

In [ ]:
def corr_simulate(Fd, Fs, N, SNR):
    # Generate channels for the specified Doppler frequency and save to file.
    wrong_packets = 0
    generate_channels_for_doppler_frequencies(Fd, Fs, N)
    with open("channels.txt", "r") as f:
        channel_vals = [json.loads(line)[0] for line in f]
    channel_vals = np.array([complex(re, im) for re, im in channel_vals])
    packet_message = np.random.randint(2, size = N)
    encoded_packet_symbols = [-1 if j == 1 else 1 for j in packet_message]
    received_symbols = transmit_data(encoded_packet_symbols, channel_vals, SNR)
    recovered_message = decoder_custom_jakes(received_symbols, channel_vals)
    for i in range(len(recovered_message)):
            if packet_message[i] != recovered_message[i]:
                wrong_packets += 1
    return np.float16(wrong_packets/N)

In [ ]:
# Define the range of SNR values to be simulated
snr_values = [round(x * 0.5, 1) for x in range(31)]
N = 100000
Fs = 1000000
Fd = 10
n1 = 3456
d_v1 = 3
d_c1 = 6
mse_threshold = 0.1

# Initialize lists to store BER for the two approaches
ber_coded_list_corr = []  # BER for the autoregressive approach
ber_uncoded_list_corr = []  # BER for the packetwise approach

# Simulation loop for each SNR value
# The simulation is peformed 3 times to ensure averaging and smoothness of the curve
for snr in snr_values:
    ber_coded_sum_corr = 0  # For autoregressive approach
    ber_uncoded_sum_corr = 0  # For packetwise approach
    for i in range(3):
        ber_coded_corr = corr_coded(Fd, Fs, N, snr, n1, d_v1, d_c1)
        ber_uncoded_corr = corr_simulate(Fd, Fs, N, snr)
        ber_coded_sum_corr += ber_coded_corr
        ber_uncoded_sum_corr += ber_uncoded_corr
    ber_coded_list_corr.append(ber_coded_sum_corr / 3)
    ber_uncoded_list_corr.append(ber_uncoded_sum_corr/ 3)


In [ ]:
plt.figure(figsize=(10, 6))
snr_values_np = np.array(snr_values)
# Plot the BER for the autoregressive approach
plt.plot(snr_values_np, ber_coded_list_corr, marker='o', linestyle='-', label='Coded Rayleigh with Jakes spectrum')
# Plot the BER for the packetwise approach
plt.plot(snr_values_np, ber_uncoded_list_corr, marker='s', linestyle='--', label='Uncoded Rayleigh with Jakes spectrum')
# Set the y-axis to a logarithmic scale
plt.yscale('log')
plt.xlabel('Eb/N0 (dB)', fontsize=14)
plt.ylabel('Bit Error Rate (BER)', fontsize=14)
plt.title('BER vs Eb/N0 for Coded and Uncoded BPSK over correlated Rayleigh channel with AWGN', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [1]:
print("Done")



print("Simulation and analysis completed successfully.")

Done
Simulation and analysis completed successfully.
